In [1]:
import pandas as pd

# Extracting Data

In [2]:
conf = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
recov = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")
deaths = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")

In [3]:
print(conf.shape, recov.shape, deaths.shape)

(279, 559) (264, 559) (279, 559)


In [4]:
conf.head(5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/20/21,7/21/21,7/22/21,7/23/21,7/24/21,7/25/21,7/26/21,7/27/21,7/28/21,7/29/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,142414,142414,143183,143439,143439,143439,144285,145008,145552,145996
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,132740,132763,132797,132828,132853,132875,132891,132922,132952,132999
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,155784,157005,158213,159563,160868,162155,163660,165204,167131,168668
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,14379,14379,14464,14498,14498,14498,14577,14586,14586,14655
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,41061,41227,41405,41629,41736,41780,41879,42110,42288,42486


### Country Scale

In [5]:
tables = (conf, recov, deaths)

for table in tables:
    
    table.drop(["Province/State", "Lat","Long"], axis = 1, inplace = True)
    
    table.rename({"Country/Region":"Country"}, axis = 1, inplace = True)

In [6]:
conf.head(5)

,Country,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,7/20/21,7/21/21,7/22/21,7/23/21,7/24/21,7/25/21,7/26/21,7/27/21,7/28/21,7/29/21
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,142414,142414,143183,143439,143439,143439,144285,145008,145552,145996
1,Albania,0,0,0,0,0,0,0,0,0,...,132740,132763,132797,132828,132853,132875,132891,132922,132952,132999
2,Algeria,0,0,0,0,0,0,0,0,0,...,155784,157005,158213,159563,160868,162155,163660,165204,167131,168668
3,Andorra,0,0,0,0,0,0,0,0,0,...,14379,14379,14464,14498,14498,14498,14577,14586,14586,14655
4,Angola,0,0,0,0,0,0,0,0,0,...,41061,41227,41405,41629,41736,41780,41879,42110,42288,42486


In [7]:
conf.shape[1]

556

##### We are supposed to be left with column counts minus 1 integers(date columns)

In [8]:
for table in tables: print(table.dtypes.value_counts(),"\n")

int64     555
object      1
dtype: int64 

int64     555
object      1
dtype: int64 

int64     555
object      1
dtype: int64 



##### Aggregation of the values into country scales

In [9]:
conf = conf.groupby("Country").sum()
recov = recov.groupby("Country").sum()
deaths = deaths.groupby("Country").sum()

In [10]:
tables = (conf, recov, deaths) 

for table in tables: print(table.shape, "\n")

(195, 555) 

(195, 555) 

(195, 555) 



### Transposing

In [11]:
conf = conf.stack().reset_index().rename({"level_1":"Date", 0 : "Confirmed"}, axis = 1)
recov = recov.stack().reset_index().rename({"level_1":"Date", 0 : "Recovered"}, axis = 1)
deaths = deaths.stack().reset_index().rename({"level_1":"Date", 0 : "Deaths"}, axis = 1)
tables = (conf, recov, deaths)

conf.head()

,Country,Date,Confirmed
0,Afghanistan,1/22/20,0
1,Afghanistan,1/23/20,0
2,Afghanistan,1/24/20,0
3,Afghanistan,1/25/20,0
4,Afghanistan,1/26/20,0


### Merging

In [12]:
for table in tables: print(table.shape, "\n")

(108225, 3) 

(108225, 3) 

(108225, 3) 



In [13]:
df0 = pd.merge(conf, recov,  how="left", left_on=["Country","Date"], right_on = ["Country","Date"])
df = pd.merge(df0, deaths,  how="left", left_on=["Country","Date"], right_on = ["Country","Date"])
df["Date"] = pd.to_datetime(df.Date)

#### Updating data of France

Data of France appeared to be faulty in the data cleaning processes(next sections). Data from Ministry of Health in France("Ministère des Solidarités et de la Santé") will be used to replace "df" starting from 3rd of April 2020. 

In [14]:
france = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/f335f9ea-86e3-4ffa-9684-93c009d5e617")

france["date"] = pd.to_datetime(france.date)

france["Country"] = "France"

france = france[["Country","date","conf", "rad","dc_tot"]]

###### Standardizing the labels from the set

In [15]:
france.rename({"date":"Date", "conf" : "Confirmed","rad" : "Recovered","dc_tot":"Deaths"}, axis = 1,inplace = True)

france.fillna(0, inplace = True)

france[["Confirmed","Recovered","Deaths"]] = france[["Confirmed","Recovered","Deaths"]].astype(int)

france = france[france["Date"] > "2020-04-02"]

df[(df["Country"] == "France") & (df["Date"] > "2020-04-02")]

df.drop(df[(df["Country"] == "France") & (df["Date"] > "2020-04-02")].index, inplace = True)

df = df.append(france, ignore_index = True)

#####  Checking the data frame after the last manipulations

In [16]:
df.isna().sum()

Country      0
Date         0
Confirmed    0
Recovered    0
Deaths       0
dtype: int64

In [17]:
df.dtypes

Country              object
Date         datetime64[ns]
Confirmed             int64
Recovered             int64
Deaths                int64
dtype: object

In [18]:
df.head(5)

,Country,Date,Confirmed,Recovered,Deaths
0,Afghanistan,2020-01-22,0,0,0
1,Afghanistan,2020-01-23,0,0,0
2,Afghanistan,2020-01-24,0,0,0
3,Afghanistan,2020-01-25,0,0,0
4,Afghanistan,2020-01-26,0,0,0


### Creating New Attributes

#### Active cases

In [19]:
df["Active Cases"] = df["Confirmed"] - (df["Recovered"] + df["Deaths"])

#### Death Rates in percentage

In [20]:
df["Death Rate(%)"] = ((df["Deaths"].divide(df["Confirmed"]))*100).fillna(0)

#### Daily Increase of Confirmed Cases

In [21]:
df = df.sort_values(["Country","Date"])
df["Increase"] = df.groupby("Country")["Confirmed"].diff().fillna(0)
df["Increase"] = df["Increase"].astype(int)

#### The day that countries reported their first case(s)

In [22]:
firstdays = df[df["Confirmed"] >= 1].groupby("Country").min().reset_index()[["Country","Date"]]
firstdays.rename({"Date":"FirstDays"}, axis = 1, inplace = True)
df = pd.merge(df, firstdays,  how="left", left_on=["Country"], right_on = ["Country"])

"Palau" has no confirmed cases in the source and causes problems with the code. Therefore, it is excluded.

In [23]:
df = df[df["Country"] != "Palau"]

#### Counts of the days passed after the first confirmed case for each country

In [24]:
df["day_xth"] = (df["Date"] - df["FirstDays"]).astype(str)

In [25]:
funct = lambda x : int(x[0:-5])

df["day_xth"] = df["day_xth"].apply(funct)

df.loc[df["day_xth"] < 0 , ["day_xth"]] = 0

#### Population

Population dataset, which was taken from United Nations, was processed to be merged with this covid-19 set

In [26]:
pop = pd.read_csv("Populations.csv")
pop = pop[["Country","Total Population(Thousands)"]]
df = pd.merge(df, pop,  how="left", left_on=["Country"], right_on = ["Country"]).fillna(0)
df.rename({"Total Population(Thousands)":"Population_1k"}, axis = 1, inplace = True)
df[["Population_1k"]] = df[["Population_1k"]].astype(int)

#### The day that countries reported at least 10 cases for the first time

In [27]:
Case10Day = df[df["Confirmed"] >= 10].groupby("Country").min().reset_index()[["Country","Date"]]
Case10Day.rename({"Date":"TenCase"}, axis = 1, inplace = True)
df = pd.merge(df, Case10Day,  how="left", left_on=["Country"], right_on = ["Country"])

#### Cumulative cases per 1,000 people

In [28]:
df["Cases/1000s"] = df["Confirmed"].divide(df["Population_1k"]).fillna(0)

In [29]:
df.head()

,Country,Date,Confirmed,Recovered,Deaths,Active Cases,Death Rate(%),Increase,FirstDays,day_xth,Population_1k,TenCase,Cases/1000s
0,Afghanistan,2020-01-22,0,0,0,0,0.0,0,2020-02-24,0,38928,2020-03-11,0.0
1,Afghanistan,2020-01-23,0,0,0,0,0.0,0,2020-02-24,0,38928,2020-03-11,0.0
2,Afghanistan,2020-01-24,0,0,0,0,0.0,0,2020-02-24,0,38928,2020-03-11,0.0
3,Afghanistan,2020-01-25,0,0,0,0,0.0,0,2020-02-24,0,38928,2020-03-11,0.0
4,Afghanistan,2020-01-26,0,0,0,0,0.0,0,2020-02-24,0,38928,2020-03-11,0.0


# Data Cleaning

### Checking the source, using anomalies

##### Counts and cumulative values of the increases where they were calculated as negative values

In [30]:
df[df["Increase"] < 0].groupby("Country").agg(["sum","count"])["Increase"].sort_values(["sum"]).head(20)

,sum,count
Country,,
France,-349612,2
Spain,-84753,3
Kazakhstan,-59375,1
Ecuador,-11066,4
United Kingdom,-7149,2
Germany,-4910,1
Czechia,-2214,1
Denmark,-2001,1
India,-1858,1


Especially in the first days of the pandemic, these negative values mostly occurred because of the obscurity of the virus and mistakes in the recording process. Sometimes, officials changed their minds and case numbers when we were less experienced.

To see the causes in this dataset; data of Spain, France and Ecuador were compared with the data from other sources. They were recorded with the same values, except France.

Errors in data of France were discovered in this step. Therefore, updated. The initial value in the count attribute for France was almost 10 days.

##### Counts and cumulative values of the active cases where they were calculated as negative values

In [31]:
df[df["Active Cases"] < 0].groupby("Country").agg(["sum","count"])["Active Cases"].sort_values(["sum"]).head(10)

,sum,count
Country,,
Peru,-19177676,283
El Salvador,-3628,4
Kosovo,-493,14
Madagascar,-170,2
Cote d'Ivoire,-62,1
Singapore,-36,1
Suriname,-18,4
Timor-Leste,-4,1
Djibouti,-3,1


Negative values in the active cases attribute most likely to show the lack/ineffectiveness of testings in a country. Infected individuals may be passing away before a confirmation.

### Checking the created attributes, comparing different calculations 

#### Exploring NaN values

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 107670 entries, 0 to 107669
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Country        107670 non-null  object        
 1   Date           107670 non-null  datetime64[ns]
 2   Confirmed      107670 non-null  int64         
 3   Recovered      107670 non-null  int64         
 4   Deaths         107670 non-null  int64         
 5   Active Cases   107670 non-null  int64         
 6   Death Rate(%)  107670 non-null  float64       
 7   Increase       107670 non-null  int32         
 8   FirstDays      107670 non-null  datetime64[ns]
 9   day_xth        107670 non-null  int64         
 10  Population_1k  107670 non-null  int32         
 11  TenCase        104340 non-null  datetime64[ns]
 12  Cases/1000s    107670 non-null  float64       
dtypes: datetime64[ns](3), float64(2), int32(2), int64(5), object(1)
memory usage: 10.7+ MB


The number of countries that never reported more than 10 cases is shown below

In [33]:
(df.groupby("Country").max()["Confirmed"] < 10).sum()

6

The number of countries that has a null value in "TenCaseDate" is shown below

In [34]:
df[df["TenCase"].isnull()].value_counts("Country").count()

6

In [35]:
df[df["TenCase"].isnull()].value_counts("Country")

Country
Kiribati            555
MS Zaandam          555
Marshall Islands    555
Micronesia          555
Samoa               555
Vanuatu             555
dtype: int64

Five countries have never reported 10 cases, hence the null values in the "TenCase" attribute(MS Zaandam is a cruise ship)

#### Comparing "Confirmed" and "Increase" 

The last values in the "Confirmed" are supposed to be equal to the sum of "Increase"

In [36]:
df2 = df.groupby(["Country"]).agg(["sum","last"])[["Confirmed","Increase"]]

df3 = df2["Confirmed"]["last"] - df2["Increase"]["sum"]

df3 = pd.DataFrame(data = df3)

df3[df3[0] !=0]

,0
Country,
China,548
Japan,2
"Korea, South",1
Taiwan*,1
Thailand,4
US,1


Increase values for the first day in the dataset can not be calculated for six countries that started to confirm cases before the first day in the dataset.

Checking to see if only the first days of these countries cause the difference; 

In [37]:
country6 = df3[df3[0] !=0].index

filtered = df[df.Country.isin(country6)]

filtered[filtered["Date"] == "2020-01-22"][["Country","Confirmed"]]

,Country,Confirmed
19980,China,548
48285,Japan,2
51060,"Korea, South",1
94905,Taiwan*,1
96570,Thailand,4
99900,US,1


Increase values were correctly calculated for the rest of the set.

Also, the "FirstDays" for these countries and "TenCase" for China must be updated.

In [38]:
df.loc[df.Country == "China", "FirstDays"] = "2019-12-31"
df.loc[df.Country == "China", "TenCase"] = "2019-12-31"
df.loc[df.Country == "Japan", "FirstDays"] = "2020-01-16"
df.loc[df.Country == "Korea, South", "FirstDays"] = "2020-01-20"
df.loc[df.Country == "Taiwan*", "FirstDays"] = "2020-01-21"
df.loc[df.Country == "Thailand", "FirstDays"] = "2020-01-08"
df.loc[df.Country == "US", "FirstDays"] = "2020-01-21"

df["FirstDays"] = pd.to_datetime(df.FirstDays)
df["TenCase"] = pd.to_datetime(df.TenCase)

Lastly, increase for the first day of these 6 countries must be updated

In [39]:
df.loc[(df["Date"] == "2020-01-22") & (df["Country"] == "China"), ["Increase"]] = None
df.loc[(df["Date"] == "2020-01-22") & (df["Country"] == "Japan"), ["Increase"]] = None
df.loc[(df["Date"] == "2020-01-22") & (df["Country"] == "Korea, South"), ["Increase"]] = None
df.loc[(df["Date"] == "2020-01-22") & (df["Country"] == "Taiwan*"), ["Increase"]] = None
df.loc[(df["Date"] == "2020-01-22") & (df["Country"] == "Thailand"), ["Increase"]] = None
df.loc[(df["Date"] == "2020-01-22") & (df["Country"] == "US"), ["Increase"]] = None

# Saving the dataset

In [40]:
df.to_csv("Covid-19 Dataset.csv", index = False)

In [41]:
df

,Country,Date,Confirmed,Recovered,Deaths,Active Cases,Death Rate(%),Increase,FirstDays,day_xth,Population_1k,TenCase,Cases/1000s
0,Afghanistan,2020-01-22,0,0,0,0,0.000000,0.0,2020-02-24,0,38928,2020-03-11,0.000000
1,Afghanistan,2020-01-23,0,0,0,0,0.000000,0.0,2020-02-24,0,38928,2020-03-11,0.000000
2,Afghanistan,2020-01-24,0,0,0,0,0.000000,0.0,2020-02-24,0,38928,2020-03-11,0.000000
3,Afghanistan,2020-01-25,0,0,0,0,0.000000,0.0,2020-02-24,0,38928,2020-03-11,0.000000
4,Afghanistan,2020-01-26,0,0,0,0,0.000000,0.0,2020-02-24,0,38928,2020-03-11,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
107665,Zimbabwe,2021-07-25,97894,65913,3094,28887,3.160561,617.0,2020-03-20,492,14862,2020-04-06,6.586866
107666,Zimbabwe,2021-07-26,99944,67827,3173,28944,3.174778,2050.0,2020-03-20,493,14862,2020-04-06,6.724802
107667,Zimbabwe,2021-07-27,101711,70496,3280,27935,3.224823,1767.0,2020-03-20,494,14862,2020-04-06,6.843695
107668,Zimbabwe,2021-07-28,103567,71383,3340,28844,3.224965,1856.0,2020-03-20,495,14862,2020-04-06,6.968578
